In [1]:
import django
import os
import json
import os.path

In [2]:
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
from boto3.session import Session
import boto3
import botocore
from loader import load_credential
import random
import string
from PIL import Image
from PIL import ExifTags
from io import BytesIO
from django.core.files.uploadedfile import InMemoryUploadedFile
import natsort

In [4]:
######### upload Clothes Image to S3 #########

from kraken.models import *

In [5]:
clothes_path = os.getenv('HOME') +'/Desktop/picture_data/clothes'
clothes_file_folder = os.listdir(clothes_path)
clothes_sorted_folder = natsort.natsorted(clothes_file_folder,reverse=False) # USE This!

In [5]:
def data_align(folder):
    """
    data 정렬하는 함수힙니다.
    ['1 (1).jpg', '1 (2).jpg']  ==>  {1: ['1 (1).jpg', '1 (2).jpg' ...], 2: ['2 (1).jpg'...] ...}
    """
    clothes_dict = {}
    k = 0
    data =[]
    for file in folder:
        name = file.split('.')[0]
        key = name.split(' ')[0]
        if key == '':
            break
        key = int(key)
        order = name.split(' ')[1].split('(')[1].split(')')[0]
        if k != key:
            data = []
        k = key
        data.append(file)
        clothes_dict[key] = data
    
    return clothes_dict

In [6]:
def ratate_image(image):
    """
    이미지 저장 시 회전되는 문제를 해결하기 위해 사용합니다.
    """
    for orientation in ExifTags.TAGS.keys():
        if ExifTags.TAGS[orientation] == 'Orientation':
            break
            
    exif = dict(image._getexif().items())
    
    if orientation in exif:
        
        if exif[orientation] == 3:
            image = image.rotate(180, expand=True)
        elif exif[orientation] == 6:
            image = image.rotate(270, expand=True)
        elif exif[orientation] == 8:
            image = image.rotate(90, expand=True)
        
    output = BytesIO()
    image.save(output, "JPEG")
    output.seek(0)
    
    return image, output

In [7]:
def save_clothes_image(aligned_data, path):
    
    kind_of = KindOf.objects.get(kinds='short_sleeve')
    active_version = DataVersion.objects.get(is_active=True)
    
    for filename in aligned_data:
        
        # tag 를 저장하기 위한 picture data 와 dataset
        old_file_name = '{} (1).jpg'.format(filename)
        dataset = Dataset.objects.create(kind_of=kind_of, old_file_name=old_file_name)
        picture_data = ClothesPicture.objects.create(version=active_version, dataset=dataset)
        
        print('===============')
        print('{} 번째 dataset 생성'.format(filename))
        print('===============')


        feature_id = 1
        for filename in aligned_data[filename]:
            feature = ClothesFeatures.objects.get(id=feature_id)
            
            byteImg = Image.open(os.path.join(path, filename))
            
            rotated_image, byteIO = ratate_image(byteImg)
            
            picture= InMemoryUploadedFile(byteIO, None, 'picture.jpg', 'image/jpeg', len(byteIO.getvalue()), None)
            
            ClothesPictureTag.objects.create(picture_data=picture_data, features=feature, picture=picture)
            
            feature_id += 1
            
            print('{} 생성완료'.format(feature))
    
    print('======END=======')
    print('데이터 업로드 완료')

In [9]:
aligned_data = data_align(clothes_sorted_folder)

In [ ]:
# save_clothes_image(aligned_data, clothes_path)

In [9]:
######### upload Wearing Image to S3 #########

In [10]:
wearing_path = os.getenv('HOME') +'/Desktop/picture_data/wearing'
wearing_file_folder = os.listdir(wearing_path)
wearing_sorted_folder = natsort.natsorted(wearing_file_folder,reverse=False) # USE This!

In [18]:
def save_wearing_image(aligned_data, path):
    
    kind_of = KindOf.objects.get(kinds='short_sleeve')
    active_version = DataVersion.objects.get(is_active=True)
    
    for filename in aligned_data:
        
        # tag 를 저장하기 위한 picture data 와 dataset
        old_file_name = '{} (1).jpg'.format(filename)
        dataset = Dataset.objects.get(old_file_name=old_file_name)
        picture_data = WearingPicture.objects.create(version=active_version, dataset=dataset)
        
        print('===============')
        print('{} 번째 dataset 생성'.format(filename))
        print('===============')


        feature_order = 0
        for filename in aligned_data[filename]:
            feature = WearingFeatures.objects.get(order=feature_order)
            
            byteImg = Image.open(os.path.join(path, filename))
            
            rotated_image, byteIO = ratate_image(byteImg)
            
            picture = InMemoryUploadedFile(byteIO, None, 'w_picture.jpg', 'image/jpeg', len(byteIO.getvalue()), None)
            
            WearingPictureTag.objects.create(picture_data=picture_data, features=feature, picture=picture)
            
            feature_order += 1
            
            print('{} 생성완료'.format(feature))
    
    print('======END=======')
    print('데이터 업로드 완료')

In [19]:
aligned_data = data_align(wearing_sorted_folder)

In [ ]:
# save_wearing_image(aligned_data, wearing_path)